# Gestor de investimento
---

In [2]:
import pandas as pd
import numpy as np
import os
import xlrd


# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetado', 'Qtd Dia', 
                                    'Qtd Garantia BOV', 'Qtd Garantia BMF', 
                                    'Qtd Estruturados', 'Liq Termo', 
                                    'Qtd Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock


def get_picking(df):
    picking = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Ação':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Renda Fixa':
            break
        elif start == True:
            picking.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    pd_picking = pd.DataFrame(columns=['Papel', 'Qtd Provisionada', 
                                    'Tipo', 'Data Pagamento', 'Valor'], data=picking)
  
    return pd_picking


def get_fi(df):
    x = []
    start = False

    for index, row in df[~df.isnull()].iterrows():
        if row['Unnamed: 2'] == 'Fundos de Investimentos':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome Fundo':
            continue
        elif row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
            break
        elif (start == True) and ((type(row['Unnamed: 2']) == str)):
            if type(row['Unnamed: 2']) == str: 
                fi_name = np.array([row['Unnamed: 2']])
                data = df[df.index == index +1][['Unnamed: 13', 'Unnamed: 19', 
                                                    'Unnamed: 33', 'Unnamed: 40', 
                                                    'Unnamed: 48', 'Unnamed: 54', 
                                                    'Unnamed: 60', 'Unnamed: 70', 
                                                    'Unnamed: 81']]
                
                data.insert(0, 'Nome FI', row['Unnamed: 2'])
                # print(data.values[0])
                x.append(data.values[0])
                #display(data.head())
    
    pd_fi = pd.DataFrame(columns=['Nome', 'Data', 'Qtd Cotas', 
                                    'Valor Cota', 'Valor Bruto', 
                                    'IR', 'IOF', 'Valor Liquido', 
                                    'Aplicacao Pendente', 'Total Bruto'], data=x)
    return pd_fi


def get_fii(df):
    fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome':
            continue
        elif row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
            break
        elif start == True:
            fii.append(row[['Unnamed: 2', 'Unnamed: 14', 
                            'Unnamed: 26', 'Unnamed: 38', 
                            'Unnamed: 45', 'Unnamed: 55', 
                            'Unnamed: 74']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetada', 'Qtd Dia', 'Qtde Total', 
                                    'Ult Cotacao', 'Financeiro'], data=fii)
  
    return df_result


def get_picking_fii(df):
    p_fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Clubes de Investimentos':
            break
        elif start == True:
            p_fii.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Tipo', 
                                    'Qtd Provisionada', 'Dt Pagamento', 
                                    'Valor Provisionado'], data=p_fii)
  
    return df_result

In [4]:
files = os.listdir('../datasets/')

stocks = pd.DataFrame()
pickings = pd.DataFrame()
fis = pd.DataFrame()
fiis = pd.DataFrame()
picking_fii = pd.DataFrame()

for file_name in files:
    if file_name == '.DS_Store':
        continue
    
    wb = xlrd.open_workbook('../datasets/' + file_name, logfile=open(os.devnull, 'w'))
    df = pd.read_excel(wb)
    
    #df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    month = int(date_position.dt.month.values)
    year = int(date_position.dt.year.values)
    period = str(year) + '/' + str(month)

    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = period
    stocks = stocks.append(df_stocks, ignore_index=True)

    df_pickings = get_picking(df)
    df_pickings['period'] = period
    pickings = pickings.append(df_pickings, ignore_index=True)

    df_fi = get_fi(df)
    df_fi['period'] = period
    fis = fis.append(df_fi, ignore_index=True)

    df_fii = get_fii(df)
    df_fii['period'] = period
    fiis = fiis.append(df_fii, ignore_index=True)

    df_picking_fii = get_picking_fii(df)
    df_picking_fii['period'] = period

    picking_fii = picking_fii.append(df_picking_fii, ignore_index=True)

# display(stocks.sort_values('period', ascending=False))
# display(fi)

In [5]:
picking_fii.sort_values(by='period')

,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
9,RNGO11,RENDIMENTO,49,15/02/2018,26.46,2018/1
27,RNGO11,RENDIMENTO,49,16/11/2018,23.52,2018/10
5,RNGO11,RENDIMENTO,49,14/12/2018,23.52,2018/11
6,RNGO11,RENDIMENTO,49,15/01/2019,23.52,2018/12
8,RNGO11,RENDIMENTO,49,14/03/2018,26.46,2018/2
13,RNGO11,RENDIMENTO,49,13/04/2018,26.46,2018/3
16,RNGO11,RENDIMENTO,49,15/05/2018,26.46,2018/4
12,RNGO11,RENDIMENTO,49,14/06/2018,26.46,2018/5
10,RNGO11,RENDIMENTO,49,16/07/2018,26.46,2018/6
21,RNGO11,RENDIMENTO,49,14/08/2018,21.56,2018/7


In [6]:
fis['period'].sort_values().unique()

array(['2018/1', '2018/10', '2018/11', '2018/12', '2018/2', '2018/3',
       '2018/4', '2018/5', '2018/6', '2018/7', '2018/8', '2018/9',
       '2019/1', '2019/10', '2019/11', '2019/12', '2019/2', '2019/3',
       '2019/4', '2019/5', '2019/6', '2019/7', '2019/8', '2019/9',
       '2020/1', '2020/2', '2020/3', '2020/4'], dtype=object)

In [7]:
display(stocks.head())
display(pickings.head())
display(fis.head())
display(fiis.head())
display(picking_fii.head())

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period
0,JHSF3,1600,0,0,0,0,0,0,1600,7.01,"11,216.00",2020/2
1,MGLU3,800,0,0,0,0,0,0,800,50.52,"40,416.00",2020/2
2,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019/9
3,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020/4
4,JHSF3,1600,0,0,0,0,0,0,1600,4.16,"6,656.00",2020/4


,Papel,Qtd Provisionada,Tipo,Data Pagamento,Valor,period
0,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020/2
1,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020/2
2,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020/4
3,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020/4
4,WEGE3,300,JUROS SOBRE CAPITAL PROPRIO,12/08/2020,9.07,2020/4


,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-02-28,"6,902.02",2.91,"20,105.88",38.48,0,"20,067.40",0,"20,105.88",2020/2
1,Equitas Selection FIC FIA,2020-02-28,"3,018.52",7.23,"21,812.68",286.90,0,"21,525.78",0,"21,812.68",2020/2
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020/2
3,Mauá Macro FIC FIM,2020-02-28,37.17,787.62,"29,278.24",186.19,0,"29,092.05",0,"29,278.24",2020/2
4,Bahia AM Maraú FIC de FIM,2020-02-28,"33,810.43",2.56,"86,466.95",67.77,0,"86,399.18",0,"86,466.95",2020/2


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period
0,IBFF11,50,0,0,50.00,100.10,"5,005.00",2020/2
1,BCFF11,8,0,0,8.00,98.17,785.36,2020/2
2,CNES11,11,0,0,11.00,83.50,918.50,2020/2
3,OULG11B,50,0,0,50.00,105.00,"5,250.00",2020/2
4,RNGO11,49,0,0,49.00,96.61,"4,733.89",2020/2


,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
0,RNGO11,RENDIMENTO,49,13/03/2020,23.03,2020/2
1,RNGO11,RENDIMENTO,49,14/10/2019,24.01,2019/9
2,RNGO11,RENDIMENTO,49,14/08/2019,24.01,2019/7
3,RNGO11,RENDIMENTO,49,15/07/2019,26.95,2019/6
4,RNGO11,RENDIMENTO,49,15/05/2020,20.58,2020/4


In [8]:
fis.head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-02-28,"6,902.02",2.91,"20,105.88",38.48,0,"20,067.40",0,"20,105.88",2020/2
1,Equitas Selection FIC FIA,2020-02-28,"3,018.52",7.23,"21,812.68",286.90,0,"21,525.78",0,"21,812.68",2020/2
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020/2
3,Mauá Macro FIC FIM,2020-02-28,37.17,787.62,"29,278.24",186.19,0,"29,092.05",0,"29,278.24",2020/2
4,Bahia AM Maraú FIC de FIM,2020-02-28,"33,810.43",2.56,"86,466.95",67.77,0,"86,399.18",0,"86,466.95",2020/2


In [9]:
fis[fis['Aplicacao Pendente'] > 0]

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
42,Polo Norte I FIC FIM,2018-12-31,"3,946.89",7.61,"30,017.36",2.60,0,"30,014.76",30000,"30,017.36",2018/12
